<a href="https://colab.research.google.com/github/gvosper25/GeorgiaVosper_3009ICT/blob/main/GeorgiaVosper_3009ICT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing and Visualization
-----
**Assignment Topic: Data Analysis on Car Advertisements**


Student Name: Georgia Vosper

SNumber: S5223182

# Data Import

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import sqlite3
import seaborn as sns
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [6]:
car_data = pd.read_csv ('/content/drive/MyDrive/3009ICT-Assignment/car_dataset.csv')
sellers_data = pd.read_csv ('/content/drive/MyDrive/3009ICT-Assignment/seller_dataset.csv')

# Data Formatting

## a. Numerical Conversion